# Crunchbase Snapshot © 2013 Data Analysis Notebook

## Import required modules.

In [53]:
%matplotlib inline
# import matplotlib.pyplot as plt
from pandas import DataFrame as df
from py2neo import authenticate, Graph, Node, Relationship
from scripts.vis import draw
import numpy as np
# import plotly as py
# from plotly.graph_objs import *

# CSS files for more aesthetically pleasing inline tables.
from IPython.core.display import HTML
css = open('css/style-table.css').read() + open('css/style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

## Import Crunchbase 2013 Snapshot CSV files.

In [285]:
cb_objects = df.from_csv('csv/cb_objects.csv').reset_index('id').fillna('')
cb_acquisitions = df.from_csv('csv/cb_acquisitions.csv').reset_index('id').fillna('')
cb_funding_rounds = df.from_csv('csv/cb_funding_rounds.csv').reset_index('id').fillna('')
cb_funds = df.from_csv('csv/cb_funds.csv').reset_index('id').fillna('')
cb_investments = df.from_csv('csv/cb_investments.csv').reset_index('id').fillna('')
cb_ipos = df.from_csv('csv/cb_ipos.csv').reset_index('id').fillna('')
cb_milestones = df.from_csv('csv/cb_milestones.csv').reset_index('id').fillna('')
cb_offices = df.from_csv('csv/cb_offices.csv').reset_index('id').fillna('')
cb_people = df.from_csv('csv/cb_people.csv').reset_index('id').fillna('')
cb_relationships = df.from_csv('csv/cb_relationships.csv').reset_index('id').fillna('')

## Visualize data frames.

In [286]:
# cb_acquisitions.head()
# cb_funding_rounds.head()
# cb_funds.head()
# cb_investments.head()
# cb_ipos.head()
# cb_milestones.head()
cb_objects.head()
# cb_offices.head()
# cb_people.head()
# cb_relationships.head()

,id,entity_type,entity_id,parent_id,name,normalized_name,permalink,category_code,status,founded_at,...,last_funding_at,funding_rounds,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships,created_by,created_at,updated_at
0,c:1,Company,1,,Wetpaint,wetpaint,/company/wetpaint,web,operating,2005-10-17,...,2008-05-19,3,3.975e+07,2010-09-05,2013-09-18,5,17,initial-importer,2007-05-25 06:51:27,2013-04-13 03:29:00
1,c:10,Company,10,,Flektor,flektor,/company/flektor,games_video,acquired,,...,,,,,,,6,initial-importer,2007-05-31 21:11:51,2008-05-23 23:23:14
2,c:100,Company,100,,There,there,/company/there,games_video,acquired,,...,,,,2003-02-01,2011-09-23,4,12,initial-importer,2007-08-06 23:52:45,2013-11-04 02:09:48
3,c:10000,Company,10000,,MYWEBBO,mywebbo,/company/mywebbo,network_hosting,operating,2008-07-26,...,,,,,,,,,2008-08-24 16:51:57,2008-09-06 14:19:18
4,c:10001,Company,10001,,THE Movie Streamer,the movie streamer,/company/the-movie-streamer,games_video,operating,2008-07-26,...,,,,,,,,,2008-08-24 17:10:34,2008-09-06 14:19:18


## Begin building the neo4j graph.

In [287]:
# Build a graph for the company objects.
objects = Graph(user = 'neo4j', password = 'hello')
objects.delete_all()

## Add companies first.

In [288]:
# Populate the graph with nodes from cb_objects data frame.
for i in range(0, len(cb_objects)):
    # Instantiate company node.
    company = Node('Company', name = cb_objects['name'][i])
    # Add node attributes.
    for j in list(cb_objects.columns.values):
        if type(cb_objects[j][i]) is np.int64:
            company[j] = int(cb_objects[j][i])
        else:
            company[j] = cb_objects[j][i]
    
    # Add the node to the objects graph.
    objects.create(company)

## Draw the (lonely) graph of companies.

In [290]:
options = {'Company': 'name'}
draw(objects, options, physics = True)

## Add people to the graph.

In [291]:
cb_people.head()

,id,object_id,first_name,last_name,birthplace,affiliation_name
0,1,p:2,Ben,Elowitz,,Blue Nile
1,2,p:3,Kevin,Flaherty,,Wetpaint
2,3,p:4,Raju,Vegesna,,Zoho
3,4,p:5,Ian,Wenig,,Zoho
4,5,p:6,Kevin,Rose,"Redding, CA",i/o Ventures


In [292]:
for i in range(0, len(cb_people)):
    # Get the person's attributes.
    try:
        object_id = cb_people['object_id'][i]
        first_name = cb_people['first_name'][i]
        last_name = cb_people['last_name'][i]
        birthplace = cb_people['birthplace'][i]
        affiliation_name = cb_people['affiliation_name'][i]
    
        person = Node('Person', 
                      name = first_name + ' ' + last_name, 
                      last_name = last_name, 
                      first_name = first_name, 
                      birthplace = birthplace, 
                      object_id = object_id,
                      affiliation_name = affiliation_name)
        
        objects.create(person)
    except:
        continue

## Draw the graph of people and companies.

In [293]:
options = {'Company': 'name', 'Person': 'name'}
draw(objects, options, physics = True)

## Build relationships between people and companies.

In [294]:
cb_relationships

,id,relationship_id,person_object_id,relationship_object_id,start_at,end_at,is_past,sequence,title,created_at,updated_at
0,1,1,p:2,c:1,,,0,8,Co-Founder/CEO/Board of Directors,2007-05-25 07:03:54,2013-06-03 09:58:46
1,2,2,p:3,c:1,,,1,279242,VP Marketing,2007-05-25 07:04:16,2010-05-21 16:31:34
2,3,3,p:4,c:3,,,0,4,Evangelist,2007-05-25 19:33:03,2013-06-29 13:36:58
3,4,4,p:5,c:3,2006-03-01,2009-12-01,1,4,Senior Director Strategic Alliances,2007-05-25 19:34:53,2013-06-29 10:25:34
4,6,6,p:7,c:4,2005-07-01,2010-04-05,1,1,Chief Executive Officer,2007-05-25 20:05:33,2010-04-05 18:41:41
5,7,7,p:8,c:4,,,1,1,Senior Software Engineer,2007-05-25 20:06:45,2010-01-12 01:13:29
6,8,8,p:9,c:4,,,1,1,Systems Engineering Manager,2007-05-25 20:07:50,2010-08-03 22:00:58
7,9,9,p:10,c:5,,,0,1,"Founder and CEO, Board Of Directors",2007-05-25 21:51:46,2010-01-25 21:49:44
8,10,10,p:11,c:5,,,1,1,Co-Founder,2007-05-25 22:15:05,2011-08-11 23:48:45
9,11,11,p:12,c:5,,,1,3,"Chief Revenue Officer, VP of Operations",2007-05-25 22:15:59,2010-01-25 21:49:45


In [295]:
for i in range(0, len(cb_relationships)):
    # Parse each row of cb_relationships frame.
    try:
        person_id = cb_relationships['person_object_id'][i]
        person = objects.find_one('Person', 
                                  property_key = 'object_id', 
                                  property_value = person_id)
        
        company_id = cb_relationships['relationship_object_id'][i]
        company = objects.find_one('Company', 
                                   property_key = 'id', 
                                   property_value = company_id)
        
        r = Relationship(person, "WORKS_FOR", company)
        
        objects.create(r)
    except:
        continue

## There are few relationships of this type in the data set.

In [297]:
options = {'Company': 'name', 'Person': 'name'}
draw(objects, options, physics = True)

## Add funds to the network.

In [256]:
cb_funds.head()

,id,fund_id,object_id,name,funded_at,raised_amount,raised_currency_code,source_url,source_description,created_at,updated_at
0,1,1,f:371,Second Fund,2008-12-16,300000000.0,USD,http://www.pehub.com/26194/dfj-dragon-raising-...,peHub,2008-12-17 03:07:16,2008-12-17 03:07:16
1,4,4,f:17,Sequoia Israel Fourth Fund,2008-12-17,200750000.0,USD,http://www.pehub.com/26725/sequoia-israel-rais...,Sequoia Israel Raises Fourth Fund,2008-12-18 22:04:42,2008-12-18 22:04:42
2,5,5,f:951,Tenth fund,2008-08-11,650000000.0,USD,http://venturebeat.com/2008/08/11/interwest-cl...,Venture Beat,2008-12-31 09:47:51,2008-12-31 09:47:51
3,6,6,f:192,New funds acquire,NaN,625000000.0,USD,http://venturebeat.com/2008/07/28/us-venture-p...,U.S. Venture Partners raises $625M fund for ne...,2009-01-01 18:13:44,2009-01-01 18:16:27
4,7,7,f:519,Third fund,2008-05-20,200000000.0,USD,http://venturebeat.com/2008/05/20/disneys-stea...,Venture Beat,2009-01-03 09:51:58,2013-09-03 16:34:54


In [250]:
for i in range(0, len(cb_funds)):
    # Get the fund's attributes.
    try:
        object_id = cb_funds['object_id'][i]
        name = cb_funds['name'][i]
        raised_amount = cb_funds['raised_amount'][i]
        source_url = cb_funds['source_url'][i]
    
        fund = Node('Fund', 
                      name = name, 
                      object_id = object_id,
                      raised_amount = raised_amount, 
                      source_url = source_url)
        
        objects.create(fund)
    except:
        continue

## Draw the graph, now with companies, people, and funds.

In [251]:
options = {'Company': 'name', 'Person': 'name', 'Fund': 'name'}
draw(objects, options, physics = True)

## Add offices.

In [277]:
cb_offices.head()

,id,object_id,office_id,description,region,address1,address2,city,zip_code,state_code,country_code,latitude,longitude,created_at,updated_at
0,1,c:1,1,,Seattle,710 - 2nd Avenue,Suite 1100,Seattle,98104,WA,USA,47.6031,-122.333,,
1,2,c:3,3,Headquarters,SF Bay,4900 Hopyard Rd,Suite 310,Pleasanton,94588,CA,USA,37.6929,-121.905,,
2,3,c:4,4,,SF Bay,135 Mississippi St,,San Francisco,94107,CA,USA,37.7647,-122.395,,
3,4,c:5,5,Headquarters,SF Bay,1601 Willow Road,,Menlo Park,94025,CA,USA,37.416,-122.152,,
4,5,c:7,7,,SF Bay,Suite 200,654 High Street,Palo Alto,94301,CA,ISR,,,,


In [300]:
for i in range(0, len(cb_offices)):
    # Get the office's attributes.
    try:
        object_id = cb_offices['object_id'][i]
        office_id = int(cb_offices['office_id'][i])
        description = cb_offices['description'][i]
        region = cb_offices['region'][i]
        zip_code = cb_offices['zip_code'][i]
        city = cb_offices['city'][i]
        state = cb_offices['state_code'][i]
        country = cb_offices['country_code'][i]
    
        office = Node('Office', 
                      object_id = object_id,
                      office_id = office_id,
                      description = description,
                      region = region,
                      zip_code = zip_code,
                      city = city,
                      state = state,
                      country = country)
    
        objects.create(office)
    except:
        continue

## Draw the graph, including offices.

In [301]:
options = {'Company': 'name', 'Person': 'name', 'Fund': 'name', 'Office': 'description'}
draw(objects, options, physics = True)

## Build edges between companies and offices.

In [298]:
cb_offices.head()

,id,object_id,office_id,description,region,address1,address2,city,zip_code,state_code,country_code,latitude,longitude,created_at,updated_at
0,1,c:1,1,,Seattle,710 - 2nd Avenue,Suite 1100,Seattle,98104,WA,USA,47.6031,-122.333,,
1,2,c:3,3,Headquarters,SF Bay,4900 Hopyard Rd,Suite 310,Pleasanton,94588,CA,USA,37.6929,-121.905,,
2,3,c:4,4,,SF Bay,135 Mississippi St,,San Francisco,94107,CA,USA,37.7647,-122.395,,
3,4,c:5,5,Headquarters,SF Bay,1601 Willow Road,,Menlo Park,94025,CA,USA,37.416,-122.152,,
4,5,c:7,7,,SF Bay,Suite 200,654 High Street,Palo Alto,94301,CA,ISR,,,,


In [307]:
for i in range(0, len(cb_offices)):
    # Parse each row of cb_offices frame.
    try:
        office_id = int(cb_offices['office_id'][i])
        person = objects.find_one('Person', 
                                  property_key = 'office_id', 
                                  property_value = office_id)
        
        company_id = cb_offices['object_id'][i]
        company = objects.find_one('Company', 
                                   property_key = 'id', 
                                   property_value = company_id)
        
        r = Relationship(company, "HAS_OFFICE", office)
        
        objects.create(r)
    except:
        continue

## Draw the company/office edges.

In [305]:
options = {'Company': 'name', 'Person': 'name', 'Fund': 'name', 'Office': 'description'}
draw(objects, options, physics = True)